In [1]:
import requests
import rasterio
from rasterio.enums import Resampling
import geopandas as gpd
from shapely.geometry import Point
from sqlalchemy import create_engine
import psycopg2

In [2]:
usgs_tiffs_url = r'https://tnmaccess.nationalmap.gov/api/v1/products?prodFormats=GeoTIFF&prodExtents=1%20x%201%20degree&datasets=National%20Elevation%20Dataset%20(NED)%201%20arc-second%20Current&polyType=state&polyCode=27&&offset=0&max=1000'

In [3]:
response = requests.get(usgs_tiffs_url)

In [4]:
urls = []
for i in range(len(response.json()['items'])):
    url = response.json()['items'][i]['urls']['TIFF']
    urls.extend([url])

In [5]:
# Define the target resolution in decimal degrees (approximately 0.008983° for ~1 km)
target_resolution = 0.008983
data = []

for i in range(len(urls)):
    with rasterio.open(urls[i]) as src:
        # Get original resolutions (pixel size) in decimal degrees
        original_resolution_x = src.transform[0]
        original_resolution_y = -src.transform[4]

        # Calculate scaling factors
        scale_x = target_resolution / original_resolution_x
        scale_y = target_resolution / original_resolution_y

        # Calculate new dimensions
        width = int(src.width / scale_x)
        height = int(src.height / scale_y)

        # Update the transform for the new resolution
        new_transform = src.transform * src.transform.scale(
            (src.width / width),
            (src.height / height)
        )

        # Perform resampling
        resampled_data = src.read(
            1,
            out_shape=(height, width),
            resampling=Resampling.average
        )
        crs = src.crs  # Preserve CRS

    for j in range(resampled_data.shape[0]):
        for k in range(resampled_data.shape[1]):
            value = resampled_data[j, k]
            x, y = rasterio.transform.xy(new_transform, j, k)
            data.append({"elevation": value, "geometry": Point(x, y)})

    print(f'Iteration {i + 1}/{len(urls)} complete')

Iteration 1/45 complete
Iteration 2/45 complete
Iteration 3/45 complete
Iteration 4/45 complete
Iteration 5/45 complete
Iteration 6/45 complete
Iteration 7/45 complete
Iteration 8/45 complete
Iteration 9/45 complete
Iteration 10/45 complete
Iteration 11/45 complete
Iteration 12/45 complete
Iteration 13/45 complete
Iteration 14/45 complete
Iteration 15/45 complete
Iteration 16/45 complete
Iteration 17/45 complete
Iteration 18/45 complete
Iteration 19/45 complete
Iteration 20/45 complete
Iteration 21/45 complete
Iteration 22/45 complete
Iteration 23/45 complete
Iteration 24/45 complete
Iteration 25/45 complete
Iteration 26/45 complete
Iteration 27/45 complete
Iteration 28/45 complete
Iteration 29/45 complete
Iteration 30/45 complete
Iteration 31/45 complete
Iteration 32/45 complete
Iteration 33/45 complete
Iteration 34/45 complete
Iteration 35/45 complete
Iteration 36/45 complete
Iteration 37/45 complete
Iteration 38/45 complete
Iteration 39/45 complete
Iteration 40/45 complete
Iteration

In [6]:
elevation = gpd.GeoDataFrame(data, crs = crs)
elevation = elevation.to_crs('EPSG:4326')

In [7]:
min_latitude = 43
max_latitude = 49.5
min_longitude = -97.5
max_longitude = -89

invalid_geom = elevation[(elevation['geometry'].y < min_latitude) |
                         (elevation['geometry'].y > max_latitude) |
                         (elevation['geometry'].x < min_longitude) |
                         (elevation['geometry'].x > max_longitude)]

if invalid_geom.empty:
    print('All geometry is valid')
else:
    print('Invalid points:')
    print(invalid_geom)
    elevation = elevation[(elevation['geometry'].y >= min_latitude) &
                          (elevation['geometry'].y <= max_latitude) &
                          (elevation['geometry'].x >= min_longitude) &
                          (elevation['geometry'].x <= max_longitude)]

Invalid points:
         elevation                    geometry
381951  463.886688  POINT (-97.99715 47.99715)
381952  463.746399  POINT (-97.98811 47.99715)
381953  462.970276  POINT (-97.97907 47.99715)
381954  463.401581  POINT (-97.97003 47.99715)
381955  463.416046  POINT (-97.96099 47.99715)
...            ...                         ...
548224  323.546265  POINT (-96.03901 49.50904)
548225  324.477081  POINT (-96.02997 49.50904)
548226  324.287720  POINT (-96.02093 49.50904)
548227  323.319092  POINT (-96.01189 49.50904)
548228  322.670898  POINT (-96.00285 49.50904)

[40265 rows x 2 columns]


In [8]:
min_elevation = 181 #lowest point in Minnesota (Lake Superior) in meters
max_elevation = 703 #highest point in Minnesota (Eagle Mountain) in meters

invalid_elevation = elevation[(elevation['elevation'] > max_elevation) | (elevation['elevation'] < min_elevation)]

if invalid_elevation.empty:
    print('All elevation values are valid')
else:
    print('Invalid elevation:')
    print(invalid_elevation)
    elevation = elevation[(elevation['elevation'] <= max_elevation) & (elevation['elevation'] >= min_elevation)]

All elevation values are valid


In [9]:
#connect to the new database to enable PostGIS
connection_string = f'postgresql://<user>:<password>@34.133.43.30:5432/lab2'
engine = create_engine(connection_string)

# Push the GeoDataFrame to PostGIS
table_name = "mn_elevation"
elevation.to_postgis(table_name, engine, if_exists="replace", index=False)

print(f"GeoDataFrame successfully pushed to the PostGIS table '{table_name}'.")

GeoDataFrame successfully pushed to the PostGIS table 'mn_elevation'.
